### **Import Libraries**

In [ ]:
import torch 
from datasets import load_dataset 
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
import numpy as np
import evaluate
from transformers import TrainingArguments, Trainer
import torch 
import matplotlib.pyplot as plt 
from transformers import DataCollatorWithPadding
import os 
from pathlib import Path
import random 
from datasets import Dataset
import warnings
from functools import partial
from datasets import concatenate_datasets, DatasetDict
from functools import partial 
from IPython.display import display
from IPython.display import Markdown
import textwrap
from transformers import pipeline
from trl import SFTTrainer
# Filter out the specific warning
warnings.filterwarnings('ignore', message='Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.')

#### **Plotting**

In [ ]:
import matplotlib as mpl
from matplotlib import pyplot as plt
from matplotlib import rcParams
rcParams['image.interpolation'] = 'nearest'
rcParams['image.cmap'] = 'viridis'
rcParams['axes.grid'] = False
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
plt.style.use('seaborn-v0_8-dark-palette')
from matplotlib import font_manager 
locations =  './../../styles/Newsreader'
font_files = font_manager.findSystemFonts(fontpaths=locations)
print(locations)
print(font_files[0])
for f in font_files: 
    font_manager.fontManager.addfont(f)
plt.rcParams["font.family"] = "Newsreader"

#### **Key Parameters**

In [ ]:
# This cell is tagged with `parameters`
model_name = 'distilbert-base-cased'
data_link = 'ppower1/instrument'
casusal_variable = False 
column = 'text'
num_epochs = 3
seed = 2 
test_size = 0.5
max_tokens = 512

#### **Helper Functions**

In [ ]:
def tokenize_function(examples, column='text'):
    return tokenizer(examples[column], truncation=True, max_length=max_tokens)

def count_tokens(example):
    tokens = tokenizer.tokenize(example['text'])
    return {"num_tokens": len(tokens)}

def count_tokens_truncated(example):
    tokens = tokenizer.tokenize(example['text'], truncation=True, max_length=max_tokens)
    return {"num_tokens": len(tokens)}


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

### ---         Memory Check
def Memory():
    print("Current memory usage:")
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')
### ---

def add_is_train_column(batch):
    # Length of the batch
    batch_size = len(batch['text'])  # Replace 'column_name' with the name of one of your columns
    # Create a list with the same value for all elements in the batch
    batch['is_train'] = [1] * batch_size
    return batch

# Modify the function for the test set
def add_is_test_column(batch):
    batch_size = len(batch['text'])  # Replace 'column_name' with the name of one of your columns
    batch['is_train'] = [0] * batch_size
    return batch

def remove_substring(example):
    # The string to be removed
    substring1 = "True or False: The Right to Counsel is in effect in the tenant's zip code."
    substring2 = "True or False: The tenant has legal representation."
    # Replace the substring with an empty string
    example["text"] = example["text"].replace(substring1, "")
    example["text"] = example["text"].replace(substring2, "")

    try:
        example["treated text"] = example["treated text"].replace(substring1, "")
        example["treated text"] = example["treated text"].replace(substring2, "")
        example["control text"] = example["control text"].replace(substring1, "")
        example["control text"] = example["control text"].replace(substring2, "")
    
    except KeyError as e:
        assert casusal_variable == False 


    return example

### ---         Print Markdown
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))
### ---

#### **Model**

In [ ]:
Memory()
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
tokenizer = AutoTokenizer.from_pretrained(model_name)
Memory()

#### **Load Data**

In [ ]:
# # Load Dataset 
# original_dataset = load_dataset(data_link)['train']
# original_dataset

seq_len, dataset_size = 512, 512
dummy_data = {
    "text": tokenizer.batch_decode(np.random.randint(100, 30000, (dataset_size, seq_len))),
    "labels": np.random.randint(0, 2, (dataset_size)),
}
original_dataset = Dataset.from_dict(dummy_data)
original_dataset.set_format("pt")

In [ ]:
with torch.no_grad():
    original_predictions = trainer.predict(ds).predictions
    original_predictions = torch.nn.functional.softmax(torch.tensor(original_predictions), dim=1)[:,1].numpy()

In [ ]:
def check_prefix(example):
    example['status'] = 1 if example['text'].startswith('Yes') else 0
    return example

def filter_status(example):
    return example['status'] == 1

# Apply the function using map
# original_dataset = original_dataset.map(check_prefix)
# original_dataset = original_dataset.filter(filter_status)
# original_dataset = original_dataset.shuffle(seed=42).select(range(1000))
# original_dataset = original_dataset.select_columns(['text', 'label'])
# original_dataset

#### **Compute Token Length**

In [ ]:
num_tokens = original_dataset.map(count_tokens)['num_tokens']
maximum_length = max(num_tokens)
print(f'Max Length:  {maximum_length}')

num_tokens_truncated = original_dataset.map(count_tokens_truncated)['num_tokens']
maximum_length_truncated = max(num_tokens_truncated)
print(f'Max Length Truncated:  {maximum_length_truncated}')

#### **Tokens Count**

In [ ]:
fig = plt.figure(dpi=300, tight_layout=True, figsize=(7, 4.5))
ax = plt.axes(facecolor=(.95, .96, .97))

# Plot customizations
for key in 'left', 'right', 'top':
    ax.spines[key].set_visible(False)
ax.text(0., 1.02, s='Count', transform=ax.transAxes, size=14)
ax.yaxis.set_tick_params(length=0)
ax.yaxis.grid(True, color='white', linewidth=2)
ax.set_axisbelow(True)
ax.set_title('Tokens in Summary of Landlord Complaint', size=14, loc='center', pad=20)

plt.hist(num_tokens, bins=50, color='#36454F')
plt.axvline(512, linestyle='--')
ax.annotate(f'Truncation',
                xy = (0.49, 0.85),
                xycoords='axes fraction',
                ha='left',
                va="center", 
                size=12)
plt.show()

#### **Clean Data**

In [ ]:
# Shuffle the dataset
original_dataset = original_dataset.shuffle(seed=3)
#original_dataset = original_dataset.shuffle(seed=42).select(range(1000))

# Remove Unnecessary Trailing String
original_dataset = original_dataset.map(remove_substring)
print(original_dataset)


#### **Train Test Split**

In [ ]:
# Split Data Set
dataset = original_dataset.train_test_split(test_size=test_size, seed=seed)

# Tokenize Dataset 
tokenized_dataset = dataset.map(tokenize_function, batched=True)

#### **Hyperparameters**

In [ ]:
metric = evaluate.load("accuracy")
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

def get_training_args(output_dir):
    return TrainingArguments(
        output_dir=output_dir,
        per_device_train_batch_size= 32,
        per_device_eval_batch_size=32,
        num_train_epochs=num_epochs,
        load_best_model_at_end=True,
        evaluation_strategy="epoch",
        logging_strategy="epoch",
        save_strategy="epoch",
        #lr_scheduler_type='' #  https://stackoverflow.com/questions/77792137/how-to-fix-the-learning-rate-for-huggingface%C2%B4s-trainer
)

#### **Evaluate Initial Training Loss**

In [ ]:
# To get the initial training loss
trainer = Trainer(model=model,
                args=get_training_args("test_trainer"),
                train_dataset=tokenized_dataset["train"],
                eval_dataset=tokenized_dataset["train"],
                compute_metrics=compute_metrics,
                data_collator=data_collator
)

init_train_eval = trainer.evaluate()
init_train_loss = init_train_eval['eval_loss']
init_train_accuracy = init_train_eval['eval_accuracy']
print(f"Initial Training Loss: {init_train_loss:.4f}    |   Initial Training Accuracy: {init_train_accuracy:.4f}")

In [ ]:
with torch.no_grad():
    original_predictions = trainer.predict(tokenized_dataset['train']).predictions
    original_predictions = torch.nn.functional.softmax(torch.tensor(original_predictions), dim=1)[:,1].numpy()
plt.hist(original_predictions, bins=50)
plt.show()

#### **Evaluate Initial Validation Loss**

In [ ]:
# Actual Trainer
model.eval()
trainer = Trainer(
                model=model,
                args=get_training_args('output_dir'),
                train_dataset=tokenized_dataset["train"],
                eval_dataset=tokenized_dataset["test"],
                compute_metrics=compute_metrics,
                data_collator=data_collator,
                tokenizer=tokenizer,
)

init_eval_eval = trainer.evaluate()
init_eval_loss = init_eval_eval['eval_loss']
init_eval_accuracy = init_eval_eval['eval_accuracy']
print(f"Initial Evaluation Loss: {init_eval_loss:.4f}    |   Initial Evaluation Accuracy: {init_eval_accuracy:.4f}")

In [ ]:
# Train
model.train()
trainer.train()

In [ ]:
steps, train_loss = [0] + [i['step'] for i in trainer.state.log_history if 'loss' in i], [init_train_loss] + [i['loss'] for i in trainer.state.log_history if 'loss' in i]
_, eval_loss = [i['step'] for i in trainer.state.log_history if 'eval_loss' in i], [init_eval_loss] + [i['eval_loss'] for i in trainer.state.log_history if 'eval_loss' in i]
eval_accuracy = [init_eval_accuracy] + [i['eval_accuracy'] for i in trainer.state.log_history if 'eval_accuracy' in i]
lr = [i['learning_rate'] for i in trainer.state.log_history if 'learning_rate' in i]

In [ ]:
plt.plot(lr)
plt.show()

In [ ]:
fig = plt.figure(dpi=300, tight_layout=True, figsize=(7, 4.5))
ax = plt.axes(facecolor=(.95, .96, .97))

# Plot customizations
for key in 'left', 'right', 'top':
    ax.spines[key].set_visible(False)
ax.text(0., 1.02, s='Count', transform=ax.transAxes, size=14)
ax.yaxis.set_tick_params(length=0)
ax.yaxis.grid(True, color='white', linewidth=2)
ax.set_axisbelow(True)
ax.set_title('Tokens in Summary of Landlord Complaint', size=14, loc='center', pad=20)

plt.plot(train_loss, label='Training Loss')
plt.plot(eval_loss, label='Validation loss')
plt.plot(eval_accuracy, label='Accuracy')

# Annotating each line at its last data point
x_positions = len(train_loss) -2  # Assuming all lists have the same length
y_pos_shift = 0.05

ax.annotate(f'Accuracy',
            xy=(x_positions, eval_accuracy[-1] + y_pos_shift),
            xytext=(8, 0),  # Shift text to right slightly
            textcoords='offset points',
            va="center", 
            ha='left',
            size=12)

ax.annotate(f'Train Loss',
            xy=(x_positions, train_loss[-1] + y_pos_shift),
            xytext=(8, 0),
            textcoords='offset points',
            va="center", 
            ha='left',
            size=12)

ax.annotate(f'Validation Loss',
            xy=(x_positions, eval_loss[-1] + y_pos_shift),
            xytext=(8, 0),
            textcoords='offset points',
            va="center", 
            ha='left',
            size=12)
plt.xlim(0, num_epochs)
plt.ylim(0, 1)
plt.show()

In [ ]:
with torch.no_grad():
    original_predictions = trainer.predict(tokenized_dataset['test']).predictions
    original_predictions = torch.nn.functional.softmax(torch.tensor(original_predictions), dim=1)[:,1].numpy()

In [ ]:
plt.hist(original_predictions, bins=20)
plt.xlim(0, 1)
plt.show()

#### **Clean Up**

In [ ]:
!rm -rf ./output_dir

In [ ]:
!rm -rf ./synth_evict/